In [16]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import config
import mimetypes
import tarfile
import io

# Your client credentials
client_id = config.client_id
client_secret = config.api_secret

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

# Get token for the session
token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                          client_secret=client_secret)


# Examples for S5PL2

Carbon Monoxide, CO (RGB visualization and transparency with dataMask)

In [17]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["CO", "dataMask"],
    output: { bands: 4 },
  }
}

const minVal = 0.0
const maxVal = 0.1
const diff = maxVal - minVal

const rainbowColors = [
  [minVal, [0, 0, 0.5]],
  [minVal + 0.125 * diff, [0, 0, 1]],
  [minVal + 0.375 * diff, [0, 1, 1]],
  [minVal + 0.625 * diff, [1, 1, 0]],
  [minVal + 0.875 * diff, [1, 0, 0]],
  [maxVal, [0.5, 0, 0]],
]

const viz = new ColorRampVisualizer(rainbowColors)

function evaluatePixel(sample) {
  var rgba = viz.process(sample.CO)
  rgba.push(sample.dataMask)
  return rgba
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "bbox": [
                13,
                45,
                15,
                47,
            ],
        },
        "data": [
            {
                "type": "sentinel-5p-l2",
                "dataFilter": {
                    "timeRange": {
                        "from": "2018-12-28T00:00:00Z",
                        "to": "2018-12-31T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_5/S5PL2_Example1.png"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


Nitrogen Dioxide, NO2 (NRTI timeliness, RGB visualization and transparency with dataMask)

In [18]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["NO2", "dataMask"],
    output: { bands: 4 },
  }
}

const minVal = 0.0
const maxVal = 0.0001
const diff = maxVal - minVal

const rainbowColors = [
  [minVal, [0, 0, 0.5]],
  [minVal + 0.125 * diff, [0, 0, 1]],
  [minVal + 0.375 * diff, [0, 1, 1]],
  [minVal + 0.625 * diff, [1, 1, 0]],
  [minVal + 0.875 * diff, [1, 0, 0]],
  [maxVal, [0.5, 0, 0]],
]

const viz = new ColorRampVisualizer(rainbowColors)

function evaluatePixel(sample) {
  var rgba = viz.process(sample.NO2)
  rgba.push(sample.dataMask)
  return rgba
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "bbox": [
                13,
                45,
                15,
                47,
            ],
        },
        "data": [
            {
                "type": "sentinel-5p-l2",
                "dataFilter": {
                    "timeRange": {
                        "from": "2018-12-30T00:00:00Z",
                        "to": "2018-12-31T00:00:00Z",
                    },
                    "timeliness": "NRTI",
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_5/S5PL2_Example2.png"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


Formaldehyde, HCHO (float32 format, specific value for no data, GeoTIFF)

In [19]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["HCHO", "dataMask"],
    output: { bands: 1, sampleType: "FLOAT32" },
  }
}

function evaluatePixel(sample) {
  if (sample.dataMask == 1) {
    return [sample.HCHO]
  } else {
    return [-9999]
  }
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "bbox": [
                13,
                45,
                15,
                47,
            ],
        },
        "data": [
            {
                "type": "sentinel-5p-l2",
                "dataFilter": {
                    "timeRange": {
                        "from": "2018-12-30T00:00:00Z",
                        "to": "2018-12-31T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request, headers={"Accept": "image/tiff"})

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_5/S5PL2_Example3.tiff"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


Ozone, O3 (RPRO timeliness, streched values and dataMask)

In [20]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["O3", "dataMask"],
    output: { bands: 2 },
  }
}

function evaluatePixel(sample, scene) {
  var maxVal = 0.36
  return [sample.O3 / maxVal, sample.dataMask]
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "bbox": [
                13,
                45,
                15,
                47,
            ],
        },
        "data": [
            {
                "type": "sentinel-5p-l2",
                "dataFilter": {
                    "timeRange": {
                        "from": "2019-04-22T00:00:00Z",
                        "to": "2019-04-23T00:00:00Z",
                    },
                    "timeliness": "RPRO",
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_5/S5PL2_Example4.png"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


Sulfur Dioxide, SO2 (minQa=20 applied, streched values)

In [21]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["SO2", "dataMask"],
    output: { bands: 2 },
  }
}

function evaluatePixel(sample, scene) {
  var maxVal = 0.01
  return [sample.SO2 / maxVal, sample.dataMask]
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "bbox": [
                13,
                45,
                15,
                47,
            ],
        },
        "data": [
            {
                "type": "sentinel-5p-l2",
                "dataFilter": {
                    "timeRange": {
                        "from": "2018-12-30T00:00:00Z",
                        "to": "2018-12-31T00:00:00Z",
                    }
                },
                "processing": {"minQa": 20},
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_5/S5PL2_Example5.png"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


Methane, CH4

In [22]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["CH4", "dataMask"],
    output: { bands: 4 },
  }
}

const minVal = 1600.0
const maxVal = 2000.0
const diff = maxVal - minVal

const rainbowColors = [
  [minVal, [0, 0, 0.5]],
  [minVal + 0.125 * diff, [0, 0, 1]],
  [minVal + 0.375 * diff, [0, 1, 1]],
  [minVal + 0.625 * diff, [1, 1, 0]],
  [minVal + 0.875 * diff, [1, 0, 0]],
  [maxVal, [0.5, 0, 0]],
]

const viz = new ColorRampVisualizer(rainbowColors)

function evaluatePixel(sample) {
  var rgba = viz.process(sample.CH4)
  rgba.push(sample.dataMask)
  return rgba
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "bbox": [
                10,
                20,
                15,
                25,
            ],
        },
        "data": [
            {
                "type": "sentinel-5p-l2",
                "dataFilter": {
                    "timeRange": {
                        "from": "2018-12-28T00:00:00Z",
                        "to": "2018-12-31T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_5/S5PL2_Example6.png"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


AER AI 340 and 380

In [23]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["AER_AI_340_380", "dataMask"],
    output: { bands: 4 },
  }
}

const minVal = -1.0
const maxVal = 5.0
const diff = maxVal - minVal

const rainbowColors = [
  [minVal, [0, 0, 0.5]],
  [minVal + 0.125 * diff, [0, 0, 1]],
  [minVal + 0.375 * diff, [0, 1, 1]],
  [minVal + 0.625 * diff, [1, 1, 0]],
  [minVal + 0.875 * diff, [1, 0, 0]],
  [maxVal, [0.5, 0, 0]],
]

const viz = new ColorRampVisualizer(rainbowColors)

function evaluatePixel(sample) {
  var rgba = viz.process(sample.AER_AI_340_380)
  rgba.push(sample.dataMask)
  return rgba
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "bbox": [
                13,
                45,
                15,
                47,
            ],
        },
        "data": [
            {
                "type": "sentinel-5p-l2",
                "dataFilter": {
                    "timeRange": {
                        "from": "2018-12-28T00:00:00Z",
                        "to": "2018-12-31T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_5/S5PL2_Example7.png"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


AER AI 354 and 388

In [24]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["AER_AI_354_388", "dataMask"],
    output: { bands: 4 },
  }
}

const minVal = -1.0
const maxVal = 5.0
const diff = maxVal - minVal

const rainbowColors = [
  [minVal, [0, 0, 0.5]],
  [minVal + 0.125 * diff, [0, 0, 1]],
  [minVal + 0.375 * diff, [0, 1, 1]],
  [minVal + 0.625 * diff, [1, 1, 0]],
  [minVal + 0.875 * diff, [1, 0, 0]],
  [maxVal, [0.5, 0, 0]],
]

const viz = new ColorRampVisualizer(rainbowColors)

function evaluatePixel(sample) {
  var rgba = viz.process(sample.AER_AI_354_388)
  rgba.push(sample.dataMask)
  return rgba
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "bbox": [
                13,
                45,
                15,
                47,
            ],
        },
        "data": [
            {
                "type": "sentinel-5p-l2",
                "dataFilter": {
                    "timeRange": {
                        "from": "2018-12-28T00:00:00Z",
                        "to": "2018-12-31T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_5/S5PL2_Example8.png"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


Cloud base height

In [25]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["CLOUD_BASE_HEIGHT", "dataMask"],
    output: { bands: 4 },
  }
}

const minVal = 0
const maxVal = 20000.0
const diff = maxVal - minVal

const rainbowColors = [
  [minVal, [0, 0, 0.5]],
  [minVal + 0.125 * diff, [0, 0, 1]],
  [minVal + 0.375 * diff, [0, 1, 1]],
  [minVal + 0.625 * diff, [1, 1, 0]],
  [minVal + 0.875 * diff, [1, 0, 0]],
  [maxVal, [0.5, 0, 0]],
]

const viz = new ColorRampVisualizer(rainbowColors)

function evaluatePixel(sample) {
  var rgba = viz.process(sample.CLOUD_BASE_HEIGHT)
  rgba.push(sample.dataMask)
  return rgba
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "bbox": [
                13,
                45,
                15,
                47,
            ],
        },
        "data": [
            {
                "type": "sentinel-5p-l2",
                "dataFilter": {
                    "timeRange": {
                        "from": "2018-12-28T00:00:00Z",
                        "to": "2018-12-31T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_5/S5PL2_Example9.png"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


Cloud base pressure

In [26]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["CLOUD_BASE_PRESSURE", "dataMask"],
    output: { bands: 4 },
  }
}

const minVal = 10000.0
const maxVal = 110000.0
const diff = maxVal - minVal

const rainbowColors = [
  [minVal, [0, 0, 0.5]],
  [minVal + 0.125 * diff, [0, 0, 1]],
  [minVal + 0.375 * diff, [0, 1, 1]],
  [minVal + 0.625 * diff, [1, 1, 0]],
  [minVal + 0.875 * diff, [1, 0, 0]],
  [maxVal, [0.5, 0, 0]],
]

const viz = new ColorRampVisualizer(rainbowColors)

function evaluatePixel(sample) {
  var rgba = viz.process(sample.CLOUD_BASE_PRESSURE)
  rgba.push(sample.dataMask)
  return rgba
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "bbox": [
                13,
                45,
                15,
                47,
            ],
        },
        "data": [
            {
                "type": "sentinel-5p-l2",
                "dataFilter": {
                    "timeRange": {
                        "from": "2018-12-28T00:00:00Z",
                        "to": "2018-12-31T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_5/S5PL2_Example10.png"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


Effective radiometric cloud fraction

In [27]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["CLOUD_FRACTION", "dataMask"],
    output: { bands: 4 },
  }
}

const minVal = 0.0
const maxVal = 1.0
const diff = maxVal - minVal

const rainbowColors = [
  [minVal, [0, 0, 0.5]],
  [minVal + 0.125 * diff, [0, 0, 1]],
  [minVal + 0.375 * diff, [0, 1, 1]],
  [minVal + 0.625 * diff, [1, 1, 0]],
  [minVal + 0.875 * diff, [1, 0, 0]],
  [maxVal, [0.5, 0, 0]],
]

const viz = new ColorRampVisualizer(rainbowColors)

function evaluatePixel(sample) {
  var rgba = viz.process(sample.CLOUD_FRACTION)
  rgba.push(sample.dataMask)
  return rgba
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "bbox": [
                13,
                45,
                15,
                47,
            ],
        },
        "data": [
            {
                "type": "sentinel-5p-l2",
                "dataFilter": {
                    "timeRange": {
                        "from": "2018-12-28T00:00:00Z",
                        "to": "2018-12-31T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_5/S5PL2_Example11.png"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


Cloud optical thickness

In [28]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["CLOUD_OPTICAL_THICKNESS", "dataMask"],
    output: { bands: 4 },
  }
}

const minVal = 0.0
const maxVal = 250.0
const diff = maxVal - minVal

const rainbowColors = [
  [minVal, [0, 0, 0.5]],
  [minVal + 0.125 * diff, [0, 0, 1]],
  [minVal + 0.375 * diff, [0, 1, 1]],
  [minVal + 0.625 * diff, [1, 1, 0]],
  [minVal + 0.875 * diff, [1, 0, 0]],
  [maxVal, [0.5, 0, 0]],
]

const viz = new ColorRampVisualizer(rainbowColors)

function evaluatePixel(sample) {
  var rgba = viz.process(sample.CLOUD_OPTICAL_THICKNESS)
  rgba.push(sample.dataMask)
  return rgba
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "bbox": [
                13,
                45,
                15,
                47,
            ],
        },
        "data": [
            {
                "type": "sentinel-5p-l2",
                "dataFilter": {
                    "timeRange": {
                        "from": "2018-12-28T00:00:00Z",
                        "to": "2018-12-31T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_5/S5PL2_Example12.png"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


Cloud top height

In [29]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["CLOUD_TOP_HEIGHT", "dataMask"],
    output: { bands: 4 },
  }
}

const minVal = 0.0
const maxVal = 20000.0
const diff = maxVal - minVal

const rainbowColors = [
  [minVal, [0, 0, 0.5]],
  [minVal + 0.125 * diff, [0, 0, 1]],
  [minVal + 0.375 * diff, [0, 1, 1]],
  [minVal + 0.625 * diff, [1, 1, 0]],
  [minVal + 0.875 * diff, [1, 0, 0]],
  [maxVal, [0.5, 0, 0]],
]

const viz = new ColorRampVisualizer(rainbowColors)

function evaluatePixel(sample) {
  var rgba = viz.process(sample.CLOUD_TOP_HEIGHT)
  rgba.push(sample.dataMask)
  return rgba
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "bbox": [
                13,
                45,
                15,
                47,
            ],
        },
        "data": [
            {
                "type": "sentinel-5p-l2",
                "dataFilter": {
                    "timeRange": {
                        "from": "2018-12-28T00:00:00Z",
                        "to": "2018-12-31T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_5/S5PL2_Example13.png"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


Cloud top pressure

In [30]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["CLOUD_TOP_PRESSURE", "dataMask"],
    output: { bands: 4 },
  }
}

const minVal = 10000.0
const maxVal = 110000.0
const diff = maxVal - minVal

const rainbowColors = [
  [minVal, [0, 0, 0.5]],
  [minVal + 0.125 * diff, [0, 0, 1]],
  [minVal + 0.375 * diff, [0, 1, 1]],
  [minVal + 0.625 * diff, [1, 1, 0]],
  [minVal + 0.875 * diff, [1, 0, 0]],
  [maxVal, [0.5, 0, 0]],
]

const viz = new ColorRampVisualizer(rainbowColors)

function evaluatePixel(sample) {
  var rgba = viz.process(sample.CLOUD_TOP_PRESSURE)
  rgba.push(sample.dataMask)
  return rgba
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "bbox": [
                13,
                45,
                15,
                47,
            ],
        },
        "data": [
            {
                "type": "sentinel-5p-l2",
                "dataFilter": {
                    "timeRange": {
                        "from": "2018-12-28T00:00:00Z",
                        "to": "2018-12-31T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_5/S5PL2_Example14.png"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved
